In [1]:
from yunet import YuNet
from sface import SFace
import glob
from sklearn import svm
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
from tqdm import tqdm
import sys
from numpy import dot
from numpy.linalg import norm

In [2]:
detector = YuNet(
    modelPath="face_detection_yunet_2023mar.onnx",
    inputSize=[320, 320],
    confThreshold=0.8,
    nmsThreshold=0.3,
    topK=5000,
)

In [3]:
recognizer = SFace(modelPath="face_recognition_sface_2021dec.onnx", disType=0)

In [4]:
def get_embeddings(person):
    face_files = glob.glob("pictures/%s/*" % person)
    # is this doing the normalization we need automatically? Check docs perhaps
    faces = [cv2.imread(f) for f in face_files]
    # get face locations
    face_descriptions = []
    for it, p in enumerate(faces):
        detector.setInputSize([p.shape[1], p.shape[0]])
        # will return None if there is no face, confirmed w tree pic
        face_extract = detector.infer(p)
        if face_extract is not None:
            face_descriptions.append([p, face_extract])

        else:
            print("No face found in %s. Continuing" % face_files[it])

    embeddings = []
    for d in face_descriptions:
        face_embedding = recognizer.infer(d[0], d[1][0][:-1])[0]
        embeddings.append(face_embedding)
    labels = [person for f in embeddings]

    return embeddings, labels

In [5]:
all_embeddings = []
all_labels = []
people = ["joey", "chandler", "ross", "rachel", "monica", "phoebe"]
for p in people:
    embeddings, labels = get_embeddings(p)
    all_embeddings.extend(embeddings)
    all_labels.extend(labels)

No face found in pictures/joey/joey_021. Continuing
No face found in pictures/rachel/rachel_020. Continuing
No face found in pictures/monica/monica_035. Continuing
No face found in pictures/monica/monica_012. Continuing
No face found in pictures/monica/monica_022. Continuing


In [6]:
clf = svm.SVC(gamma="scale", probability=True)
clf.fit(all_embeddings, all_labels)
clf.classes_

array(['chandler', 'joey', 'monica', 'phoebe', 'rachel', 'ross'],
      dtype='<U8')

In [16]:
input_movie = cv2.VideoCapture("Friendstester.mp4")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
fps = input_movie.get(cv2.CAP_PROP_FPS)
width = input_movie.get(cv2.CAP_PROP_FRAME_WIDTH)
height = input_movie.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(width, height)

640.0 480.0


In [8]:
detector.setInputSize([int(width), int(height)])

In [9]:
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
# output_movie = cv2.VideoWriter('output.mp4', fourcc, 29.97, (640, 360))

output_movie = cv2.VideoWriter(
    "output_yunet.mp4", fourcc, fps, (int(width), int(height))
)

In [10]:
#THIS IS THE ORIGINAL
frame_number = 0

for i in tqdm(range(length)):
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    # https://stackoverflow.com/questions/75926662/face-recognition-problem-with-face-encodings-function
    rgb_frame = np.ascontiguousarray(frame[:, :, ::-1])

    # Find all the faces and face encodings in the current frame of video
    face_extract = detector.infer(rgb_frame)
    if face_extract is not None:
        face_embeddings = [
            recognizer.infer(rgb_frame, face_extract[i][:-1])[0]
            for i in range(len(face_extract))
        ]
    else:
        print("No face found. Continuing")
        output_movie.write(frame)
        continue

    face_names = []
    for e in face_embeddings:
        # See if the face is a match for the known face(s)
        prediction = clf.predict_proba([e])[0]
        confidence = prediction[np.argmax(prediction)]
        label = clf.classes_[np.argmax(prediction)] + str(confidence)
        name = clf.classes_[np.argmax(prediction)] #ADDED
        print(label)
        print(name) #ADDED

        face_names.append(label)

    # Label the results
    for i in range(len(face_extract)):
        coords = face_extract[i][:-1].astype(np.int32)
        # draw border
        cv2.rectangle(
            frame,
            (coords[0], coords[1]),
            (coords[0] + coords[2], coords[1] + coords[3]),
            (0, 255, 0),
            2,
        )
        # cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(
            frame,
            (coords[0], coords[1] + coords[3] - 25),
            (coords[0] + coords[2], coords[1] + coords[3]),
            (0, 0, 255),
            cv2.FILLED,
        )
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(
            frame,
            face_names[i],
            (coords[0] + 6, coords[1] + coords[3] - 6),
            font,
            0.5,
            (255, 255, 255),
            1,
        )

    # Write the resulting image to the output video file
    output_movie.write(frame)

# All done!
input_movie.release()
output_movie.release()
cv2.destroyAllWindows()
        

  0%|▏                                         | 4/1075 [00:00<00:28, 37.19it/s]

No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing


  1%|▍                                        | 11/1075 [00:00<00:20, 51.95it/s]

No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing


  2%|▋                                        | 17/1075 [00:00<00:20, 51.28it/s]

...

monica0.900114350225424
monica
ross0.43742750319399143
ross
monica0.9018556828495122
monica
ross0.4349130387066402
ross
monica0.8926471533994177
monica
ross0.4474206799542659
ross
monica0.8926437071174324
monica
ross0.4671966482971691
ross
monica0.9048465875830829
monica
ross0.4583457244904673
ross
monica0.8791577589214397
monica
ross0.7098095324853863
ross


100%|██████████████████████████████████████▉| 1073/1075 [00:30<00:00, 27.65it/s]

monica0.9067970840710952
monica
ross0.6163301159652045
ross
monica0.9200253953247175
monica
ross0.6438215553829679
ross
monica0.9137133173990543
monica
ross0.6344088640509032
ross
monica0.8846620310697298
monica
ross0.7245705560978373
ross
monica0.9054616394080007
monica
ross0.7275740491134317
ross
monica0.9101273443292648
monica
ross0.7131308897454989
ross


100%|███████████████████████████████████████| 1075/1075 [00:30<00:00, 35.52it/s]

monica0.9033587631276412
monica
ross0.7144375310829341
ross


In [17]:
#ATTEMPT #2 HERE>>>>>>>>>>>
import pandas as pd 

frame_number = 0

face_recog_confidence = 0.7

frame_storage = []

for i in tqdm(range(length)):
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    # https://stackoverflow.com/questions/75926662/face-recognition-problem-with-face-encodings-function
    rgb_frame = np.ascontiguousarray(frame[:, :, ::-1])

    # Find all the faces and face encodings in the current frame of video
    face_extract = detector.infer(rgb_frame)
    if face_extract is not None:
        face_embeddings = [
            recognizer.infer(rgb_frame, face_extract[i][:-1])[0]
            for i in range(len(face_extract))
        ]
    else:
        print("No face found. Continuing")
        frame_storage.append([i, 0, 0, 0, 0, 0, 0])
        continue

    face_names = []
    for e in face_embeddings:
        # See if the face is a match for the known face(s)
        prediction = clf.predict_proba([e])[0]
        confidence = prediction[np.argmax(prediction)]
        # threshold confidence
        if confidence < face_recog_confidence:
            continue
            
        label = clf.classes_[np.argmax(prediction)] + str(confidence)
        name = clf.classes_[np.argmax(prediction)]
        print(label)
        print(name)
        
        face_names.append(name)
        
    # binary flag indicating whether or not a person is on the screen
    face_present = [int(x in face_names) for x in clf.classes_]
    to_store = [i]
    to_store.extend(face_present)
    
    frame_storage.append(to_store)

labels = ['frame_num']
labels.extend(clf.classes_)
out_df = pd.DataFrame(frame_storage, columns = labels)
out_df.to_csv('frame_nums.csv')

  1%|▍                                        | 11/1075 [00:00<00:20, 51.23it/s]

No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing


  3%|█                                        | 28/1075 [00:00<00:14, 70.18it/s]

No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing
No face found. Continuing


  4%|█▋                                       | 44/1075 [00:00<00:14, 73.01it/s]

...

monica0.9328382995341864
monica
monica0.8967811908818378
monica
monica0.8986799391612647
monica
monica0.9424114182653764
monica
monica0.900114350225424
monica
monica0.9018556828495122
monica


 99%|██████████████████████████████████████▋| 1067/1075 [00:27<00:00, 30.13it/s]

monica0.8926471533994177
monica
monica0.8926437071174324
monica
monica0.9048465875830829
monica
monica0.8791577589214397
monica
ross0.7098095324853863
ross
monica0.9067970840710952
monica
monica0.9200253953247175
monica


100%|███████████████████████████████████████| 1075/1075 [00:27<00:00, 39.40it/s]

monica0.9137133173990543
monica
monica0.8846620310697298
monica
ross0.7245705560978373
ross
monica0.9054616394080007
monica
ross0.7275740491134317
ross
monica0.9101273443292648
monica
ross0.7131308897454989
ross
monica0.9033587631276412
monica
ross0.7144375310829341
ross


In [18]:
##ATTEMPT #3 analyzing by seconds rather than frames 

import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm


# Video frame rate and length
frame_rate = int(input_movie.get(cv2.CAP_PROP_FPS))
video_length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

frame_number = 0
face_recog_confidence = 0.7
frame_storage = []

# Increment in seconds
time_interval = 1  # Change this to the desired time interval in seconds

# Check if frame_rate * time_interval is non-zero before entering the loop
if frame_rate * time_interval != 0:

    for i in tqdm(range(video_length // (frame_rate * time_interval))):
        # Set the video capture position to the desired time (in seconds)
        target_time = i * time_interval
        input_movie.set(cv2.CAP_PROP_POS_MSEC, target_time * 1000)

        # Grab a frame at the desired time
        ret, frame = input_movie.read()
        frame_number += 1

        # Quit when the input video file ends
        if not ret:
            break

        # Append the data to frame_storage as needed
            # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        # https://stackoverflow.com/questions/75926662/face-recognition-problem-with-face-encodings-function
        rgb_frame = np.ascontiguousarray(frame[:, :, ::-1])

        # Find all the faces and face encodings in the current frame of video
        face_extract = detector.infer(rgb_frame)
        if face_extract is not None:
            face_embeddings = [
                recognizer.infer(rgb_frame, face_extract[i][:-1])[0]
                for i in range(len(face_extract))
            ]
        else:
            print("No face found. Continuing")
            frame_storage.append([i, 0, 0, 0, 0, 0, 0])
            continue

        face_names = []
        for e in face_embeddings:
            # See if the face is a match for the known face(s)
            prediction = clf.predict_proba([e])[0]
            confidence = prediction[np.argmax(prediction)]
            # threshold confidence
            if confidence < face_recog_confidence:
                continue

            label = clf.classes_[np.argmax(prediction)] + str(confidence)
            name = clf.classes_[np.argmax(prediction)]
            print(label)
            print(name)

            face_names.append(name)

        # binary flag indicating whether or not a person is on the screen
        face_present = [int(x in face_names) for x in clf.classes_]
        to_store = [i]
        to_store.extend(face_present)

        frame_storage.append(to_store)
else:
    print("Error: frame_rate * time_interval is zero. Adjust the values.")


# Save the results to a DataFrame and CSV file
labels = ['time (s)']  # Change this label to reflect time instead of frame number
labels.extend(clf.classes_)
out_df = pd.DataFrame(frame_storage, columns=labels)
out_df.to_csv('time_intervals.csv')


 11%|████▉                                       | 2/18 [00:00<00:01, 14.08it/s]

No face found. Continuing


 44%|███████████████████▌                        | 8/18 [00:00<00:00, 16.58it/s]

chandler0.7220765323524696
chandler
chandler0.8533350651053833
chandler
monica0.9656558029344469
monica
monica0.9584082667728111
monica


 67%|████████████████████████████▋              | 12/18 [00:00<00:00, 15.89it/s]

monica0.9180821697891212
monica
joey0.9086145378789798
joey
ross0.8883586549178852
ross
monica0.9809881270084732
monica
ross0.9013399898752398
ross


 94%|████████████████████████████████████████▌  | 17/18 [00:00<00:00, 18.65it/s]

ross0.8839556712534512
ross
monica0.8048417762011415
monica
joey0.9183384853985262
joey
joey0.9500287540228535
joey
joey0.9434175945712242
joey


100%|███████████████████████████████████████████| 18/18 [00:01<00:00, 16.42it/s]

joey0.9231752322496903
joey


In [13]:
frame_storage = []
test = frame_storage.append(1)
test

In [14]:
img1 = cv2.imread("../pictures/monica/monica_001")
img2 = cv2.imread("../pictures/monica/monica_004")
img3 = cv2.imread("../pictures/test_pics/trees.jpg")
img4 = cv2.imread("../pictures/test_pics/test_joey_monica.jpeg")
img5 = cv2.imread("../pictures/test_pics/monica_side_test.png")

[ WARN:0@40.691] global loadsave.cpp:248 findDecoder imread_('../pictures/monica/monica_001'): can't open/read file: check file path/integrity
[ WARN:0@40.692] global loadsave.cpp:248 findDecoder imread_('../pictures/monica/monica_004'): can't open/read file: check file path/integrity
[ WARN:0@40.692] global loadsave.cpp:248 findDecoder imread_('../pictures/test_pics/trees.jpg'): can't open/read file: check file path/integrity
[ WARN:0@40.692] global loadsave.cpp:248 findDecoder imread_('../pictures/test_pics/test_joey_monica.jpeg'): can't open/read file: check file path/integrity
[ WARN:0@40.692] global loadsave.cpp:248 findDecoder imread_('../pictures/test_pics/monica_side_test.png'): can't open/read file: check file path/integrity


In [15]:
detector.setInputSize([img5.shape[1], img5.shape[0]])
face5 = detector.infer(img5)
face5

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
detector.setInputSize([img4.shape[1], img4.shape[0]])
face4 = detector.infer(img4)
face4

In [ ]:
detector.setInputSize([img1.shape[1], img1.shape[0]])
face1 = detector.infer(img1)
face1

In [ ]:
detector.setInputSize([img2.shape[1], img2.shape[0]])

In [ ]:
face2 = detector.infer(img2)
face2

In [ ]:
result = recognizer.match(img1, face1[0][:-1], img2, face2[0][:-1])
result

In [ ]:
feature1 = recognizer.infer(img1, face1[0][:-1])[0]
feature1

In [ ]:
feature2 = recognizer.infer(img2, face2[0][:-1])[0]
feature2

In [ ]:
cos_sim = dot(feature1, feature2) / (norm(feature1) * norm(feature2))
cos_sim

In [ ]:
cos_sim = dot(feature1, feature2) / (norm(feature1) * norm(feature2))
cos_sim